In [1]:
!pip install opendatasets

In [2]:
#for text embedding
!pip install sentence-transformers

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: saikatjana12
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis


100%|██████████| 1.99M/1.99M [00:00<00:00, 733MB/s]

In [4]:
import pandas as pd
df=pd.read_csv("/content/twitter-entity-sentiment-analysis/twitter_training.csv")
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [5]:
df['Positive'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [6]:
df=df.drop(['2401','Borderlands'],axis=1)
df.head()

,Positive,"im getting on borderlands and i will murder you all ,"
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [7]:
df=df.rename(columns={
    'Positive':'sentiment',
    'im getting on borderlands and i will murder you all ,':'comment'
})

In [8]:
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import DataLoader, TensorDataset

In [15]:
from sklearn.model_selection import train_test_split

In [9]:
level_encoder=LabelEncoder()
df['sentiment']=level_encoder.fit_transform(df['sentiment'])

In [10]:
df['comment'] = df['comment'].astype(str)

In [11]:
print(df['comment'].apply(type).value_counts())

comment
<class 'str'>    74681
Name: count, dtype: int64


In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2')
comments_list = df['comment'].tolist()
comment_embeddings = model.encode(comments_list)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
df['embedding'] = list(comment_embeddings)
df=df.drop(['comment'],axis=1)

In [16]:
x_train, x_test, y_train, y_test=train_test_split(df['embedding'],df['sentiment'],random_state=42,test_size=0.2)

In [17]:
X_train_stacked = np.array(x_train.tolist())
X_test_stacked = np.array(x_test.tolist())

In [18]:
X_train_tensor = torch.tensor(X_train_stacked, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_stacked, dtype=torch.float32)

In [19]:
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [20]:
def scaled_dot_product(q, k, v, mask=None):
    """Calculates the scaled dot-product attention."""
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiHeadAttention(nn.Module):
    """Multi-Head Attention mechanism."""
    def __init__(self, d_model, num_heads):
        super().__init__()
        # Check to prevent architectural errors before model creation
        assert d_model % num_heads == 0, f"d_model ({d_model}) must be divisible by num_heads ({num_heads})"

        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model, 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).contiguous()

        # Now we can safely reshape
        values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)

        out = self.linear_layer(values)
        return out

In [21]:
class LayerNormalization(nn.Module):
    """Layer Normalization module."""
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

In [22]:
class PositionwiseFeedForward(nn.Module):
    """Position-wise Feed-Forward network."""
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [23]:
class EncoderLayer(nn.Module):
    """A single layer of the Transformer Encoder."""
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, mask=None):
        residual_x = x
        x = self.attention(x, mask=mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x

In [24]:
class Encoder(nn.Module):
    """The full Transformer Encoder."""
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])
    def forward(self, x, mask=None):
        x = self.layers(x)
        return x


In [25]:
class SentimentClassifier(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, num_classes):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
        self.classifier_head = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # Reshape input for the encoder: (batch_size, d_model) -> (batch_size, 1, d_model)
        x = x.unsqueeze(1)
        encoder_output = self.encoder(x)
        # Squeeze output for the classifier head: (batch_size, 1, d_model) -> (batch_size, d_model)
        encoder_output = encoder_output.squeeze(1)
        logits = self.classifier_head(encoder_output)
        return logits

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [27]:
D_MODEL = 384  # This MUST match your data's feature dimension
NUM_HEADS = 8
FFN_HIDDEN = D_MODEL * 4  # A common practice
DROP_PROB = 0.1
NUM_LAYERS = 6 # Number of EncoderLayers

# Training parameters
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-4
NUM_CLASSES=4

In [28]:
torch.unique(y_train_tensor)

tensor([0, 1, 2, 3])

In [29]:
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [30]:
model = SentimentClassifier(
        d_model=D_MODEL,
        ffn_hidden=FFN_HIDDEN,
        num_heads=NUM_HEADS,
        drop_prob=DROP_PROB,
        num_layers=NUM_LAYERS,
        num_classes=NUM_CLASSES
    ).to(device)

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [32]:
print("\n Starting Training...")

# --- Step 4: The Training Loop ---
for epoch in range(EPOCHS):
    model.train()  # Set the model to training mode

    # Initialize tracking variables
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # --- Loop over batches ---
    for batch_inputs, batch_labels in train_loader:
        # Move data to the selected device (GPU/CPU)
        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        # 1. Zero the gradients
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(batch_inputs)

        # 3. Compute loss
        loss = criterion(outputs, batch_labels)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

        # --- Track statistics ---
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += batch_labels.size(0)
        correct_predictions += (predicted == batch_labels).sum().item()

    # --- End of Epoch ---
    avg_loss = total_loss / len(train_loader)
    accuracy = (correct_predictions / total_samples) * 100

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")

print("\n Training Finished!")


🚀 Starting Training...
Epoch [1/10] Loss: 1.0191 | Accuracy: 57.05%
Epoch [2/10] Loss: 0.7701 | Accuracy: 69.31%
Epoch [3/10] Loss: 0.4892 | Accuracy: 81.23%
Epoch [4/10] Loss: 0.3099 | Accuracy: 88.18%
Epoch [5/10] Loss: 0.2238 | Accuracy: 91.35%
Epoch [6/10] Loss: 0.1773 | Accuracy: 93.03%
Epoch [7/10] Loss: 0.1509 | Accuracy: 93.91%
Epoch [8/10] Loss: 0.1369 | Accuracy: 94.45%
Epoch [9/10] Loss: 0.1221 | Accuracy: 95.03%
Epoch [10/10] Loss: 0.1172 | Accuracy: 95.17%

✅ Training Finished!


In [33]:
def get_sentence_embedding(sentence, embedding_model):
    """
    Converts a raw sentence into a 384-dimension numerical vector
    using a Sentence Transformer model.
    """
    # The model.encode() method returns a NumPy array.
    embedding = embedding_model.encode(sentence)
    return embedding

In [34]:
def predict_sentiment(text, classifier_model, embedding_model, class_names, device):
    """
    Predicts the sentiment of a given text using the trained model.

    Args:
        text (str): The input comment to analyze.
        classifier_model (nn.Module): Your trained SentimentClassifier model.
        embedding_model (SentenceTransformer): The model to convert text to vectors.
        class_names (dict): A dictionary mapping class indices to names (e.g., {0: 'Negative'}).
        device (str): The device to run inference on ('cuda' or 'cpu').

    Returns:
        tuple: A tuple containing the predicted class name (str) and the
               confidence score (float).
    """
    # Set the model to evaluation mode (disables dropout, etc.)
    classifier_model.eval()

    # 1. PREPROCESS: Convert the text to its numerical representation.
    embedding = get_sentence_embedding(text, embedding_model)

    # 2. FORMAT FOR PYTORCH:
    #    a) Convert numpy array to a PyTorch tensor.
    #    b) Add a batch dimension (shape [384] -> [1, 384]).
    #    c) Move the tensor to the correct device.
    input_tensor = torch.tensor(embedding).unsqueeze(0).to(device)

    # 3. PREDICT:
    #    Perform prediction without calculating gradients to save memory and speed.
    with torch.no_grad():
        # Get the model's raw output scores (logits).
        logits = classifier_model(input_tensor)

    # 4. INTERPRET:
    #    a) Convert raw logits to probabilities using the softmax function.
    #    b) Find the highest probability and its corresponding class index.
    probabilities = F.softmax(logits, dim=1)
    confidence, predicted_class_idx = torch.max(probabilities, dim=1)

    # Map the predicted index to its human-readable class name.
    predicted_class_name = class_names[predicted_class_idx.item()]

    # Return the final result.
    return predicted_class_name, confidence.item()


In [ ]:
'Positive', 'Neutral', 'Negative', 'Irrelevant'

In [37]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
CLASS_NAMES = {2: 'Negative', 1: 'Neutral', 0: 'Positive',3:'Irrelevant'}
test_comments = [
        "that was the first borderlands session in a long time where i actually had a really satisfying combat experience. i got some really good kills",
        "im getting on borderlands and i will kill you all,",
        "Man Gearbox really needs to fix this dissapointing drops in the completely new Borderlands 3 Days DLC i cant e be fine having to be farm bosses on Mayhem 10 to e get 1 legendary foot drop while anywhere else i get 6 - 10 drops.. It Really sucks to alot",
        "<unk> Gearbox really time to fix this 10 drops in the new Borderlands 3 DLC or be fine to force bosses on Mayhem 10 to get a legendary drop while everyone else i get 6-10 drops.. Really needs alot"
    ]
print("\n--- Running Predictions ---")
for comment in test_comments:
  # This now uses your actual trained model.
  sentiment, confidence = predict_sentiment(comment, model, embedding_model, CLASS_NAMES, device)

  print(f"Comment: '{comment}'")
  print(f"--> Predicted Sentiment: {sentiment} (Confidence: {confidence:.2%})\n")


--- Running Predictions ---
Comment: 'that was the first borderlands session in a long time where i actually had a really satisfying combat experience. i got some really good kills'
--> Predicted Sentiment: Irrelevant (Confidence: 99.99%)

Comment: 'im getting on borderlands and i will kill you all,'
--> Predicted Sentiment: Irrelevant (Confidence: 99.98%)

Comment: 'Man Gearbox really needs to fix this dissapointing drops in the completely new Borderlands 3 Days DLC i cant e be fine having to be farm bosses on Mayhem 10 to e get 1 legendary foot drop while anywhere else i get 6 - 10 drops.. It Really sucks to alot'
--> Predicted Sentiment: Neutral (Confidence: 99.99%)

Comment: '<unk> Gearbox really time to fix this 10 drops in the new Borderlands 3 DLC or be fine to force bosses on Mayhem 10 to get a legendary drop while everyone else i get 6-10 drops.. Really needs alot'
--> Predicted Sentiment: Neutral (Confidence: 99.99%)



In [38]:
from google.colab import drive
import os

In [40]:
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
GDRIVE_PATH = '/content/drive/MyDrive/encoder_model'
if not os.path.exists(GDRIVE_PATH):
    os.makedirs(GDRIVE_PATH)
MODEL_SAVE_PATH = os.path.join(GDRIVE_PATH, 'sentiment_model.pth')
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f"Model saved successfully to: {MODEL_SAVE_PATH}")

Model saved successfully to: /content/drive/MyDrive/encoder_model/sentiment_model.pth


In [42]:
model_to_load = SentimentClassifier(
        d_model=D_MODEL,
        ffn_hidden=FFN_HIDDEN,
        num_heads=NUM_HEADS,
        drop_prob=DROP_PROB,
        num_layers=NUM_LAYERS,
        num_classes=NUM_CLASSES
    )

In [ ]:
#model_to_load = SentimentClassifier()

# 2. Load the saved weights from your Google Drive
#    The MODEL_SAVE_PATH is the same as defined above
model_to_load.load_state_dict(torch.load(MODEL_SAVE_PATH))

# 3. Set the model to evaluation mode
#    This is important for inference as it disables layers like Dropout
model_to_load.eval()

print("\nModel loaded successfully and is ready for prediction!")